In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [2]:
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vigoo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vigoo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


82 documents
19 classes ['ML_about', 'abt_1', 'chatbot_about', 'error_1', 'error_2', 'error_3', 'goodbye', 'greeting', 'movie_star', 'name_ask', 'options', 'queries', 'questions_2', 'questions_imp', 'response_groc', 'response_rest', 'response_weather', 'thanks', 'turing']
94 unique lemmatized words ["'s", ',', 'a', 'about', 'actor', 'alan', 'alexa', 'anyone', 'are', 'assistant', 'awesome', 'be', 'bye', 'can', 'chatbot', 'chatting', 'could', 'current', 'day', 'do', 'emotion', 'favourite', 'feel', 'feeling', 'find', 'for', 'good', 'goodbye', 'google', 'grocery', 'grocerystore', 'grocry', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'is', 'know', 'later', 'learning', 'machine', 'me', 'ml', 'my', 'name', 'near', 'next', 'nice', 'offered', 'open', 'provide', 'restarurants', 'restauranat', 'restaurant', 'restaurnt', 'resturnt', 'resurant', 'robot', 'see', 'siri', 'ssup', 'store', 'stre', 'superstar', 'support', 'temperature', 'thank', 'thanks', 'that', 'the', 't

In [3]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [5]:
model = Sequential()
print ("Type 1 for 1 hidden layer, Type 2 for 2 hidden layers and type 3 for 3 hidden layers")
user_input=input()
if '1' in user_input:
    model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(train_y[0]), activation='softmax'))
    # Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    #fitting and saving the model
    hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
    model.save('chatbot_model.h5', hist)
elif '2' in user_input:
    model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(len(train_y[0]), activation='softmax'))
    # Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    #fitting and saving the model
    hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)
    model.save('chatbot_model.h5', hist)
elif '3' in user_input:
    model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(len(train_y[0]), activation='softmax'))
    # Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    #fitting and saving the model
    hist = model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=5, verbose=1)
    model.save('chatbot_model.h5', hist)
else:
    print("Invalid input")
    quit()

print("model created")

Type 1 for 1 hidden layer, Type 2 for 2 hidden layers and type 3 for 3 hidden layers
3
Epoch 1/1000
82/82 [==============================] - 0s 5ms/step - loss: 2.9433 - accuracy: 0.0244
Epoch 2/1000
82/82 [==============================] - 0s 267us/step - loss: 2.9043 - accuracy: 0.1220
Epoch 3/1000
82/82 [==============================] - 0s 231us/step - loss: 2.8854 - accuracy: 0.0976
Epoch 4/1000
82/82 [==============================] - 0s 255us/step - loss: 2.8256 - accuracy: 0.1585
Epoch 5/1000
82/82 [==============================] - 0s 268us/step - loss: 2.7999 - accuracy: 0.1585
Epoch 6/1000
82/82 [==============================] - 0s 255us/step - loss: 2.7185 - accuracy: 0.2317
Epoch 7/1000
82/82 [==============================] - 0s 267us/step - loss: 2.6115 - accuracy: 0.2317
Epoch 8/1000
82/82 [==============================] - 0s 255us/step - loss: 2.5169 - accuracy: 0.2683
Epoch 9/1000
82/82 [==============================] - 0s 255us/step - loss: 2.2846 - accuracy: 0.30

82/82 [==============================] - 0s 280us/step - loss: 0.0633 - accuracy: 0.9878
Epoch 80/1000
82/82 [==============================] - 0s 268us/step - loss: 0.0441 - accuracy: 0.9878
Epoch 81/1000
82/82 [==============================] - 0s 243us/step - loss: 0.0181 - accuracy: 1.0000
Epoch 82/1000
82/82 [==============================] - 0s 268us/step - loss: 0.0253 - accuracy: 1.0000
Epoch 83/1000
82/82 [==============================] - 0s 243us/step - loss: 0.0086 - accuracy: 1.0000
Epoch 84/1000
82/82 [==============================] - 0s 231us/step - loss: 0.0179 - accuracy: 1.0000
Epoch 85/1000
82/82 [==============================] - 0s 231us/step - loss: 0.0224 - accuracy: 1.0000
Epoch 86/1000
82/82 [==============================] - 0s 268us/step - loss: 0.0116 - accuracy: 1.0000
Epoch 87/1000
82/82 [==============================] - 0s 243us/step - loss: 0.0505 - accuracy: 0.9878
Epoch 88/1000
82/82 [==============================] - 0s 231us/step - loss: 0.0810 - a

82/82 [==============================] - 0s 231us/step - loss: 0.0916 - accuracy: 0.9878
Epoch 159/1000
82/82 [==============================] - 0s 243us/step - loss: 0.0480 - accuracy: 0.9878
Epoch 160/1000
82/82 [==============================] - 0s 255us/step - loss: 0.0713 - accuracy: 0.9878
Epoch 161/1000
82/82 [==============================] - 0s 243us/step - loss: 0.0040 - accuracy: 1.0000
Epoch 162/1000
82/82 [==============================] - 0s 231us/step - loss: 0.0095 - accuracy: 1.0000
Epoch 163/1000
82/82 [==============================] - 0s 244us/step - loss: 0.0082 - accuracy: 1.0000
Epoch 164/1000
82/82 [==============================] - 0s 243us/step - loss: 0.0065 - accuracy: 1.0000
Epoch 165/1000
82/82 [==============================] - 0s 267us/step - loss: 0.0462 - accuracy: 0.9756
Epoch 166/1000
82/82 [==============================] - 0s 292us/step - loss: 0.2438 - accuracy: 0.9878
Epoch 167/1000
82/82 [==============================] - 0s 353us/step - loss: 0

82/82 [==============================] - 0s 255us/step - loss: 0.0644 - accuracy: 0.9634
Epoch 237/1000
82/82 [==============================] - 0s 243us/step - loss: 0.0569 - accuracy: 0.9756
Epoch 238/1000
82/82 [==============================] - 0s 243us/step - loss: 0.0507 - accuracy: 0.9878
Epoch 239/1000
82/82 [==============================] - 0s 255us/step - loss: 0.0111 - accuracy: 1.0000
Epoch 240/1000
82/82 [==============================] - 0s 353us/step - loss: 0.0191 - accuracy: 1.0000
Epoch 241/1000
82/82 [==============================] - 0s 255us/step - loss: 0.0115 - accuracy: 0.9878
Epoch 242/1000
82/82 [==============================] - 0s 243us/step - loss: 0.0238 - accuracy: 0.9878
Epoch 243/1000
82/82 [==============================] - 0s 268us/step - loss: 0.0359 - accuracy: 0.9878
Epoch 244/1000
82/82 [==============================] - 0s 255us/step - loss: 0.0070 - accuracy: 1.0000
Epoch 245/1000
82/82 [==============================] - 0s 243us/step - loss: 0

82/82 [==============================] - 0s 243us/step - loss: 0.0067 - accuracy: 1.0000
Epoch 315/1000
82/82 [==============================] - 0s 255us/step - loss: 0.0074 - accuracy: 1.0000
Epoch 316/1000
82/82 [==============================] - 0s 280us/step - loss: 0.0010 - accuracy: 1.0000
Epoch 317/1000
82/82 [==============================] - 0s 268us/step - loss: 0.0501 - accuracy: 0.9756
Epoch 318/1000
82/82 [==============================] - 0s 243us/step - loss: 0.0031 - accuracy: 1.0000
Epoch 319/1000
82/82 [==============================] - 0s 231us/step - loss: 0.0063 - accuracy: 1.0000
Epoch 320/1000
82/82 [==============================] - 0s 231us/step - loss: 0.0029 - accuracy: 1.0000
Epoch 321/1000
82/82 [==============================] - 0s 232us/step - loss: 0.0059 - accuracy: 1.0000
Epoch 322/1000
82/82 [==============================] - 0s 231us/step - loss: 0.0105 - accuracy: 1.0000
Epoch 323/1000
82/82 [==============================] - 0s 231us/step - loss: 0

...

82/82 [==============================] - 0s 218us/step - loss: 0.1130 - accuracy: 0.9756
Epoch 627/1000
82/82 [==============================] - 0s 231us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 628/1000
82/82 [==============================] - 0s 231us/step - loss: 0.1025 - accuracy: 0.9878
Epoch 629/1000
82/82 [==============================] - 0s 230us/step - loss: 0.1142 - accuracy: 0.9634
Epoch 630/1000
82/82 [==============================] - 0s 255us/step - loss: 0.0058 - accuracy: 1.0000
Epoch 631/1000
82/82 [==============================] - 0s 304us/step - loss: 4.9733e-04 - accuracy: 1.0000
Epoch 632/1000
82/82 [==============================] - 0s 256us/step - loss: 0.0377 - accuracy: 0.9756
Epoch 633/1000
82/82 [==============================] - 0s 292us/step - loss: 0.0024 - accuracy: 1.0000
Epoch 634/1000
82/82 [==============================] - 0s 243us/step - loss: 1.0815e-04 - accuracy: 1.0000
Epoch 635/1000
82/82 [==============================] - 0s 231us/step -

82/82 [==============================] - 0s 218us/step - loss: 0.0478 - accuracy: 0.9634
Epoch 705/1000
82/82 [==============================] - 0s 218us/step - loss: 0.0235 - accuracy: 0.9878
Epoch 706/1000
82/82 [==============================] - 0s 255us/step - loss: 0.0243 - accuracy: 0.9878
Epoch 707/1000
82/82 [==============================] - 0s 218us/step - loss: 0.0084 - accuracy: 1.0000
Epoch 708/1000
82/82 [==============================] - 0s 206us/step - loss: 0.0096 - accuracy: 1.0000
Epoch 709/1000
82/82 [==============================] - 0s 219us/step - loss: 0.0075 - accuracy: 1.0000
Epoch 710/1000
82/82 [==============================] - 0s 219us/step - loss: 0.0188 - accuracy: 0.9878
Epoch 711/1000
82/82 [==============================] - 0s 243us/step - loss: 0.0021 - accuracy: 1.0000
Epoch 712/1000
82/82 [==============================] - 0s 219us/step - loss: 0.0544 - accuracy: 0.9756
Epoch 713/1000
82/82 [==============================] - 0s 328us/step - loss: 0

82/82 [==============================] - 0s 256us/step - loss: 0.1258 - accuracy: 0.9756
Epoch 783/1000
82/82 [==============================] - 0s 219us/step - loss: 0.0078 - accuracy: 1.0000
Epoch 784/1000
82/82 [==============================] - 0s 244us/step - loss: 0.0629 - accuracy: 0.9878
Epoch 785/1000
82/82 [==============================] - 0s 279us/step - loss: 0.1766 - accuracy: 0.9756
Epoch 786/1000
82/82 [==============================] - 0s 268us/step - loss: 0.0368 - accuracy: 0.9756
Epoch 787/1000
82/82 [==============================] - 0s 268us/step - loss: 0.0303 - accuracy: 0.9878
Epoch 788/1000
82/82 [==============================] - 0s 268us/step - loss: 0.0126 - accuracy: 1.0000
Epoch 789/1000
82/82 [==============================] - ETA: 0s - loss: 3.5047e-06 - accuracy: 1.00 - 0s 243us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 790/1000
82/82 [==============================] - 0s 267us/step - loss: 0.0354 - accuracy: 0.9878
Epoch 791/1000
82/82 [===========

82/82 [==============================] - 0s 207us/step - loss: 0.0870 - accuracy: 0.9878
Epoch 861/1000
82/82 [==============================] - 0s 230us/step - loss: 0.0028 - accuracy: 1.0000
Epoch 862/1000
82/82 [==============================] - 0s 231us/step - loss: 0.0471 - accuracy: 0.9756
Epoch 863/1000
82/82 [==============================] - 0s 219us/step - loss: 0.4055 - accuracy: 0.9878
Epoch 864/1000
82/82 [==============================] - 0s 219us/step - loss: 0.1637 - accuracy: 0.9634
Epoch 865/1000
82/82 [==============================] - 0s 206us/step - loss: 0.0438 - accuracy: 0.9878
Epoch 866/1000
82/82 [==============================] - 0s 219us/step - loss: 0.0064 - accuracy: 1.0000
Epoch 867/1000
82/82 [==============================] - 0s 219us/step - loss: 0.0712 - accuracy: 0.9756
Epoch 868/1000
82/82 [==============================] - 0s 243us/step - loss: 7.2302e-04 - accuracy: 1.0000
Epoch 869/1000
82/82 [==============================] - 0s 231us/step - los

82/82 [==============================] - 0s 206us/step - loss: 5.4151e-04 - accuracy: 1.0000
Epoch 939/1000
82/82 [==============================] - 0s 243us/step - loss: 8.0806e-04 - accuracy: 1.0000
Epoch 940/1000
82/82 [==============================] - 0s 243us/step - loss: 0.1325 - accuracy: 0.9634
Epoch 941/1000
82/82 [==============================] - 0s 243us/step - loss: 0.0223 - accuracy: 0.9878
Epoch 942/1000
82/82 [==============================] - 0s 304us/step - loss: 0.0158 - accuracy: 0.9878
Epoch 943/1000
82/82 [==============================] - 0s 292us/step - loss: 0.0094 - accuracy: 1.0000
Epoch 944/1000
82/82 [==============================] - 0s 292us/step - loss: 0.0068 - accuracy: 1.0000
Epoch 945/1000
82/82 [==============================] - 0s 255us/step - loss: 0.0082 - accuracy: 1.0000
Epoch 946/1000
82/82 [==============================] - 0s 243us/step - loss: 1.6237e-04 - accuracy: 1.0000
Epoch 947/1000
82/82 [==============================] - 0s 255us/st